In [ ]:
import mlrun
from sklearn.datasets import load_breast_cancer

no_k8s = False if mlrun.mlconf.namespace else True

In [ ]:
project = mlrun.get_or_create_project("sta-lab9", context="./")

In [ ]:
project.set_function("data_prep.py", "data-prep", image="mlrun/mlrun")
project.set_function("trainer.py", "trainer", handler="train", image="mlrun/mlrun")
project.set_function("serving.py", "serving", image="mlrun/mlrun", kind="serving")
project.save()

In [ ]:
gen_data_run = project.run_function("data-prep", params={"format": "csv"}, local=True)
print("Data-prep run state:", gen_data_run.state())
print("Data-prep outputs:", gen_data_run.outputs)

# Display the first few rows of the dataset logged in the "dataset" artifact
df = gen_data_run.artifact("dataset").as_df()
print("Dataset preview:")
print(df.head())

In [ ]:
describe = mlrun.import_function('hub://describe')
describe_run = describe.run(
    name="task-describe",
    handler='analyze',
    inputs={"table": gen_data_run.outputs["dataset"]},
    params={"label_column": "label"},
    local=False
)
print("Describe function outputs:", describe_run.outputs)
describe_run.artifact("histograms").show()
describe_run.artifact("correlation").show()

In [ ]:
trainer_run = project.run_function(
    'trainer',
    inputs={"dataset": gen_data_run.outputs["dataset"]},
    params={"n_estimators": 100, "learning_rate": 1e-1, "max_depth": 5},
    local=True
)
print("Trainer run outputs:", trainer_run.outputs)
trainer_run.artifact("confusion-matrix").show()
trainer_run.artifact("feature-importance").show()

In [ ]:
# Step 4: Hyperparameter Tuning
hp_tuning_run = project.run_function(
    "trainer",
    inputs={"dataset": gen_data_run.outputs["dataset"]},
    hyperparams={
        "n_estimators": [10, 100, 200],
        "max_depth": [2, 5, 10]
    },
    selector="max.accuracy",
    local=False
)
print("Hyperparameter tuning outputs:", hp_tuning_run.outputs)
hp_tuning_run.artifact('parallel_coordinates').show()

In [ ]:
models = project.list_models()
for model in models:
    print(f"Model URI: {model.uri}, Metrics: {model.metrics}")

print("\nComparing trainer runs:")
project.list_runs(name="trainer-train", iter=True).compare()

In [ ]:
serving_fn = mlrun.code_to_function("serving", filename="serving.py", image="mlrun/mlrun", kind="serving")
serving_fn.add_model("rf-model", model_path=hp_tuning_run.outputs["model"], class_name="RFModel")
serving_fn.spec.graph.plot()


In [ ]:
serving_fn.spec.build.registry = "index.docker.io/dakshata123"
serving_fn.spec.build.secret = "registry-credentials"

mlrun.deploy_function(serving_fn)
print("Model deployed successfully.")

In [ ]:
data = load_breast_cancer(as_frame=True)
sample_inputs = data.frame.drop(columns=["target"]).iloc[:3].values.tolist()
my_data = {"inputs": sample_inputs}
response = serving_fn.invoke("/v2/models/rf-model/infer", body=my_data)
print("Predictions from the deployed model:", response)

In [ ]:
run_id = project.run(
    workflow_path="workflow.py",
    arguments={"model_name": "rf-model"},
    watch=True,
    local=no_k8s
)
print("Workflow run id:", run_id.id)